In [ ]:
from jetson import Camera
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from ../../vae.vae import VAE

In [ ]:
IMAGE_CHANNELS = 3
VARIANTS_SIZE = 32
MODEL_PATH = '../../vae.torch'
device = 'cuda'

In [ ]:
vae = VAE(image_channels=IMAGE_CHANNELS, z_dim=VARIANTS_SIZE)
vae.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
vae.to(torch.device(device)).eval()

In [ ]:
camera = Camera.instance(width=320, height=240)

In [ ]:
def preprocess(image):
    pass

def tensor_to_jpeg(tensor):
    pass

In [ ]:
image = widgets.Image(format='jpeg', width=320, height=240)
result = widgets.Image(format='jpeg', width=160, height=80)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image,reconst]))

In [ ]:
def vae_process(change):
    image = change['new']
    image = preprocess(image)
    z, _ ,_ = vae.encode(image)
    reconst = vae.decode(z)
    result.value = tensor_to_jpeg(reconst[0]) 

vae_process({'new': camera.value})
camera.observe(vae_process, names='value')

In [ ]:
camera.unobserve(vae_process, names='value')
camera_link.unlink()
